In [1]:
suppressPackageStartupMessages(library("igraph"))
suppressPackageStartupMessages(library("QUBIC"))
suppressPackageStartupMessages(library("QUBICdata"))
suppressPackageStartupMessages(library("biclust"))

suppressPackageStartupMessages(library('fabia'))

suppressPackageStartupMessages(library("eisa"))
suppressPackageStartupMessages(library("isa2"))

In [2]:
# path to directories with input data
exprs_dir <- "../simulated_datasets/exprs/"
network_dir <- "../simulated_datasets/networks/"
# suffix and prefix of input file name
fname_prefix <- "simulated.N=10.Mu=2.0.GxP="
fname_suffix <- ".overlap=TT.exprs.tsv"
# bicluster sizes tested
g_sizes <- c(5,10,20,50,100)
p_sizes <- c(10,20,50,100)

# where to save output files
out_dir <- "../simulated_datasets/results/"

# expected number of biclusters
expected_N =  10 
# number of runs for non-deterministic methods
n_runs = 10

In [3]:
exprs_file_name <- "simulated.N=10.Mu=2.0.GxP=10,50.overlap=TT.exprs.tsv"
exprs <- as.matrix(read.table(paste0(exprs_dir ,exprs_file_name),sep="\t", row.names=1, header=TRUE))
colnames(exprs) <- seq(0,dim(exprs)[2]-1)
basename <- gsub(".exprs.tsv","",exprs_file_name)

network_file <- paste0(network_dir,basename,".tab")

network_file
head(exprs,3)

[1] "../simulated_datasets/networks/simulated.N=10.Mu=2.0.GxP=10,50.overlap=TT.tab"

,0,1,2,3,4,5,6,7,8,9,⋯,190,191,192,193,194,195,196,197,198,199
0,1.18666913,0.8342043,-0.3426233,0.2482811,-1.6911819,-0.8704147,1.379088,-0.02256812,-0.7781378,0.6101195,⋯,0.46378773,-2.0326053,1.1595554,-0.07583626,1.6970123,-0.8061087,-0.003583603,0.2423194,-0.2122606,-0.1006038
1,0.62779701,-0.1929154,0.7512584,-0.0792025,-0.3199999,0.5813120,1.303577,0.14290491,0.4268464,-0.5297245,⋯,-1.17010286,0.6225605,-0.4007707,0.69126101,-0.1430504,0.7869674,-0.317057268,0.4417518,-0.4519059,-1.4813961
2,-0.02321444,0.2295602,0.1828850,-0.7850687,0.9299874,-0.1837959,3.422745,-0.03739051,-1.8285225,-0.3477797,⋯,0.08762657,2.1736162,-0.2601291,-2.93386414,0.7381668,0.3651644,1.065739335,-0.5488517,-1.1934198,0.3838835


## QUBIC
#### Hyperparameters
 * method method = BCQU
     * "BCQU" accepts a matrix with real values  
     * "BCQUD" (data discretized as integer); discretization - makes sense only for BCQUD
 * r [1] - the range of possible ranks; r <= n_patients/2; **r=(1,100,step=10)** *[observation: fails for high r when q is low]* == discretization_level
 * q [0.06] - percentage of the regulating conditions for each gene; **q=(0.05,0.5,step=0.05)**
 * c [0.95] - consistency level of a bicluster -??? 
 * f [1] -  ensure that no two reported biclusters overlap more than f ??
 * k [max(2,ncol(exprs) %/% 20)] - minimum column width of the block
 * type ['default'] - the constrain type used
     * 'default' -  maximizes the minimal value of numbers of rows and columns ???
     * 'area' - maximizes the value of number of rows multiplied by number of columns ???
 * P [FALSE] - whether to enlarge the bicluster until a p-value constrain reached; **P=TRUE,FALSE**
 * C [FALSE] - whether to set the lower bound of the condition number in a bicluster as 5% ; C=TRUE,FALSE
 * with (Query-based mode) or without network
 * o - expected numer of biclusters - set exauls to the number of implanted biclusters

In [ ]:
?BCQU()

In [ ]:
runQUBIC <- function(exprs,basename,out_dir="./",network_file=NULL,discretization=NULL,r=1,q=0.05,c=0.8,P=FALSE,o=100){

    if (P) {P_ = "T"}
    else {P_="F"}
    
    # this is to make filenames distinguishible
    params <- paste0("r=",r,",q=",q,",c=",c,",P=",P_)
    
    if(!is.null(network_file)) {
        network <- read.graph(network_file, format = "ncol")
        weights <- get.adjacency(network, attr = "weight") # adjacency matrix
        out_file <- paste0(out_dir,basename,".QUBIC_QB.biclust_results.txt") # query-based
    }
    else{
        network<-NULL
        weights<-NULL
        out_file <- paste0(out_dir,basename,".QUBIC.biclust_results.txt") # query-based
    }
    
    t0 <- as.numeric(Sys.time())
    results <- biclust(exprs, o=o,weight = weights, verbose = FALSE,
                        method = BCQU(), r = r, q = q, P = P, c=c)
    runtime <- as.numeric(Sys.time()) - t0
    runtime <- round(runtime,2)
    
    description <- paste0("Sample_id:",basename,"; Parameters:",params,"; Runtime:",runtime)
    biclust::writeBiclusterResults(out_file, results, description,
                               rownames(exprs), colnames(exprs), append=FALSE, delimiter="\t")
}

#runQUBIC(exprs,basename,network_file=NULL,out_dir = out_dir,o=expected_N)

In [ ]:
#  r = 1, q = 0.06, c = 0.95, o = 100
r_values <- c(1,2,3,4,5,7,10) # 
q_values <- c(0.06,0.05,0.1,0.25,0.5,0.75) # seq(0.05,0.5,0.1)
c_values <- c(0.95,0.75,0.6,0.65,0.5,0.8,0.85,0.9)
f_values <- c(1,0.5,0.25)
P_values <- c(TRUE,FALSE)


tool_out_dir <- paste0(out_dir,"QUBIC/")
if (!file.exists(tool_out_dir)){
    dir.create(file.path(out_dir,"QUBIC/"))
}

for (n_genes in  c(10,20,50,100)){
    for (n_pats in p_sizes){
        # read input data
        exprs_file_name <- paste0(fname_prefix,n_genes,",",n_pats,fname_suffix)
        exprs <- as.matrix(read.table(paste0(exprs_dir ,exprs_file_name),sep="\t", row.names=1, header=TRUE))
        colnames(exprs) <- seq(0,dim(exprs)[2]-1)
        basename <- gsub(".exprs.tsv","",exprs_file_name)
        network_file <- paste0(network_dir,basename,".tab")
        
        cat(basename ,"\n")
        flush.console()
        Sys.sleep(1)

        for (r in r_values){ 
            for (q in q_values){ 
                for (c in c_values){
                    for (P in P_values){
                        if (P) { P_="T"}
                        else { P_="F"} 
                        #cat(paste0("\tr=",r,"; q=",q,"; c=",c),"\n")
                        flush.console()
                        Sys.sleep(1)
                        params_outdir <- paste0("r=",r,".q=",q,".c=",c,".P=",P_,"/")
                        if (!file.exists(paste0(tool_out_dir,params_outdir))){
                            dir.create(file.path(tool_out_dir,params_outdir))
                        }
                        try(runQUBIC(exprs,basename,network_file=NULL,out_dir = paste0(tool_out_dir,params_outdir),
                                     r=r,q=q,P=P,c=c,o=expected_N))
                        try(runQUBIC(exprs,basename,network_file=network_file,out_dir = paste0(tool_out_dir,params_outdir),
                                     r=r,q=q,P=P,c=c,o=expected_N))
                    }
                }
            }
        }
    
    }
}

##  Plaid
#### Hyperparameters
 * row.release [0.7] - ;  [0.5-0.7] - recommended; row_release = (0.5,0.8,step =0.05)
 * col.release [0.7] - ; col_release = (0.5,0.8,step =0.1)
 * iter.startup [5] - Number of iterations to find starting values
 * iter.layer [10] - Number of iterations to find each layer
 * back.fit [0] - After a layer is added, additional iterations can be done to refine the fitting of the layer (default set to 0)
 * max.layers [20] - Maximum number of layer to include in the mode

In [4]:
runPlaid <- function(exprs, basename,out_dir="./",row.release =0.7, col.release = 0.7, 
                     back.fit = 0, max.layers = 20, iter.startup = 5, iter.layer = 10,n_runs = 10){
    append <- FALSE
    # write output
    out_file <- paste0(out_dir,basename,".Plaid.biclust_results.txt")
    params <- paste0("row.release=",row.release,",col.release=",col.release,
                     ",back.fit=",back.fit,",max.layers=",max.layers,
                    ",iter.startup=",iter.startup,",iter.layer=",iter.layer)
    
    for (i in seq(1,n_runs)){

        t0 <- as.numeric(Sys.time())
        result <- biclust(exprs, method=BCPlaid(), cluster="b",verbose = FALSE, max.layers = max.layers,
                          row.release =row.release, col.release = col.release, back.fit = back.fit, 
                          iter.startup = iter.startup, iter.layer = iter.layer) 
        runtime <- as.numeric(Sys.time()) - t0
        runtime <- round(runtime,2)

        description <- paste0("Sample_id:",basename,"; Parameters:",params,"; Runtime:",runtime,"; Run:",i)
        if (result@Number>0){
            biclust::writeBiclusterResults(out_file, result, description,
                                       rownames(exprs), colnames(exprs), append=append, delimiter="\t")
            append <- TRUE
        }
        #else{
        #    print(paste0("\t\tno biclusters found in run ",i ))
        #}
    }
}

runPlaid(exprs, basename,out_dir = out_dir,max.layers = expected_N,row.release =0.5, col.release = 0.7, 
                     back.fit = 10, iter.startup = 100, iter.layer = 1000, n_runs = n_runs)


In [5]:
row_col_release_values <- seq(0.5,0.7,0.05)
back_fit_values <- c(0,10,100)
iter_startup_values <- c(5,10,100,1000)
iter_layer_values <- c(10,100,200,500,1000)

tool_out_dir <- paste0(out_dir,"Plaid/")

if (!file.exists(tool_out_dir)){
    dir.create(file.path(out_dir,"Plaid/"))
}

for (n_genes in g_sizes){
    for (n_pats in p_sizes){
        # read input data
        exprs_file_name <- paste0(fname_prefix,n_genes,",",n_pats,fname_suffix)
        exprs <- as.matrix(read.table(paste0(exprs_dir ,exprs_file_name),sep="\t", row.names=1, header=TRUE))
        colnames(exprs) <- seq(0,dim(exprs)[2]-1)
        basename <- gsub(".exprs.tsv","",exprs_file_name)
        network_file <- paste0(network_dir,basename,".tab")    
        message(basename ,"\n")
        flush.console()
        Sys.sleep(1)
        for (row_release in row_col_release_values ){ 
            for (col_release in row_col_release_values ){ 
                for (back_fit in back_fit_values){
                    for (iter_startup in iter_startup_values){
                        for (iter_layer in iter_layer_values){
                            params_outdir <- paste0("row_release=",row_release,".col_release=",col_release,
                                                    ".back_fit=",back_fit,".iter_startup=",iter_startup,
                                                    ".iter_layer=",iter_layer,"/")
                            
                            cat(paste0("\trow.release=",row_release ,"; col.release=",col_release,
                                       "; back_fit=",back_fit,"; iter_startup=",iter_startup,"; iter_layer=",iter_layer ),"\n")
                            flush.console()
                            Sys.sleep(1)
                            
                            if (!file.exists(paste0(tool_out_dir,params_outdir))){
                                dir.create(file.path(tool_out_dir,params_outdir))
                            }
                            try(runPlaid(exprs, basename,out_dir = paste0(tool_out_dir,params_outdir),
                                     max.layers = expected_N,row.release = row_release, col.release = col_release,
                                     back.fit = back_fit, iter.startup = iter_startup, iter.layer = iter_layer,
                                     n_runs = n_runs))
                        }
                    }
                }
                
            }
        }
    
    }
}

simulated.N=10.Mu=2.0.GxP=5,10.overlap=TT



	row.release=0.5; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=10 
	row.release=0.5; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=100 
	row.release=0.5; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=200 
	row.release=0.5; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=500 
	row.release=0.5; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=1000 
	row.release=0.5; col.release=0.5; back_fit=10; iter_startup=5; iter_layer=10 


Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 100000)”

	row.release=0.5; col.release=0.5; back_fit=10; iter_startup=5; iter_layer=100 


Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 100000)”

...

	row.release=0.55; col.release=0.7; back_fit=10; iter_startup=5; iter_layer=10 
	row.release=0.55; col.release=0.7; back_fit=10; iter_startup=5; iter_layer=100 
	row.release=0.55; col.release=0.7; back_fit=10; iter_startup=5; iter_layer=200 
	row.release=0.55; col.release=0.7; back_fit=10; iter_startup=5; iter_layer=500 
	row.release=0.55; col.release=0.7; back_fit=10; iter_startup=5; iter_layer=1000 
	row.release=0.55; col.release=0.7; back_fit=100; iter_startup=5; iter_layer=10 
	row.release=0.55; col.release=0.7; back_fit=100; iter_startup=5; iter_layer=100 
	row.release=0.55; col.release=0.7; back_fit=100; iter_startup=5; iter_layer=200 


Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 100000)”

	row.release=0.55; col.release=0.7; back_fit=100; iter_startup=5; iter_layer=500 
	row.release=0.55; col.release=0.7; back_fit=100; iter_startup=5; iter_layer=1000 
	row.release=0.6; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=10 
	row.release=0.6; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=100 
	row.release=0.6; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=200 
	row.release=0.6; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=500 
	row.release=0.6; col.release=0.5; back_fit=0; iter_startup=5; iter_layer=1000 
	row.release=0.6; col.release=0.5; back_fit=10; iter_startup=5; iter_layer=10 
	row.release=0.6; col.release=0.5; back_fit=10; iter_startup=5; iter_layer=100 
	row.release=0.6; col.release=0.5; back_fit=10; iter_startup=5; iter_layer=200 
	row.release=0.6; col.release=0.5; back_fit=10; iter_startup=5; iter_layer=500 
	row.release=0.6; col.release=0.5; back_fit=10; iter_startup=5; iter_layer=1000 
	row.release=0.6; col.release=0.5; back_

	row.release=0.65; col.release=0.55; back_fit=100; iter_startup=5; iter_layer=10 
	row.release=0.65; col.release=0.55; back_fit=100; iter_startup=5; iter_layer=100 
	row.release=0.65; col.release=0.55; back_fit=100; iter_startup=5; iter_layer=200 
	row.release=0.65; col.release=0.55; back_fit=100; iter_startup=5; iter_layer=500 
	row.release=0.65; col.release=0.55; back_fit=100; iter_startup=5; iter_layer=1000 
	row.release=0.65; col.release=0.6; back_fit=0; iter_startup=5; iter_layer=10 
	row.release=0.65; col.release=0.6; back_fit=0; iter_startup=5; iter_layer=100 
	row.release=0.65; col.release=0.6; back_fit=0; iter_startup=5; iter_layer=200 
	row.release=0.65; col.release=0.6; back_fit=0; iter_startup=5; iter_layer=500 
	row.release=0.65; col.release=0.6; back_fit=0; iter_startup=5; iter_layer=1000 
	row.release=0.65; col.release=0.6; back_fit=10; iter_startup=5; iter_layer=10 
	row.release=0.65; col.release=0.6; back_fit=10; iter_startup=5; iter_layer=100 
	row.release=0.65; col.r

	row.release=0.7; col.release=0.65; back_fit=10; iter_startup=5; iter_layer=200 
	row.release=0.7; col.release=0.65; back_fit=10; iter_startup=5; iter_layer=500 
	row.release=0.7; col.release=0.65; back_fit=10; iter_startup=5; iter_layer=1000 
	row.release=0.7; col.release=0.65; back_fit=100; iter_startup=5; iter_layer=10 
	row.release=0.7; col.release=0.65; back_fit=100; iter_startup=5; iter_layer=100 
	row.release=0.7; col.release=0.65; back_fit=100; iter_startup=5; iter_layer=200 
	row.release=0.7; col.release=0.65; back_fit=100; iter_startup=5; iter_layer=500 
	row.release=0.7; col.release=0.65; back_fit=100; iter_startup=5; iter_layer=1000 
	row.release=0.7; col.release=0.7; back_fit=0; iter_startup=5; iter_layer=10 
	row.release=0.7; col.release=0.7; back_fit=0; iter_startup=5; iter_layer=100 
	row.release=0.7; col.release=0.7; back_fit=0; iter_startup=5; iter_layer=200 
	row.release=0.7; col.release=0.7; back_fit=0; iter_startup=5; iter_layer=500 
	row.release=0.7; col.release=0

## biclust XMotifs
 - depends very much on discretization level - works for 5 but not for 3 or 10
 - ns Number of columns choosen.
 - nd Number of repetitions.
 - sd Sample size in repetitions
 - alpha  - Scaling factor for column result. Padiha et al used 0.1 and 0.05. ?Min. fraction of columns?
 - number Number of bicluster to be found.

In [ ]:
runXMotifs <- function(exprs, basename,out_dir="./",discr_levels=2, ns=10,nd=10,sd=5,alpha=0.05,number=10,n_runs = 10){
    exprs_discr <- discretize(exprs,nof=discr_levels,quant=TRUE)
    
    params <- paste0("ns=",ns,",nd=",nd,",sd=",sd,",alpha=",alpha,
                        ",discr_levels=",discr_levels)
    # write output
    out_file <- paste0(out_dir,basename,".XMotifs.biclust_results.txt")
    append <- FALSE
    for (i in seq(1,n_runs)){

        t0 <- as.numeric(Sys.time())
        result <- biclust(exprs_discr, method=BCXmotifs(), ns=ns, nd=nd, sd=sd, alpha=alpha, number=number) 
        runtime <- as.numeric(Sys.time()) - t0
        runtime <- round(runtime,2)

        
        description <- paste0("Sample_id:",basename,"; Parameters:",params,"; Runtime:",runtime,"; Run:",i)
        if (result@Number>0){
            biclust::writeBiclusterResults(out_file, result, description,
                                       rownames(exprs), colnames(exprs), append=append, delimiter="\t")
            append <- TRUE
        }
        else{
            print(paste0("\t\tno biclusters found in run ",i ))
        }
    }
}

runXMotifs(exprs, basename,out_dir=out_dir,discr_levels=5,ns=150,nd=10,sd=5,alpha=0.15,n_runs = n_runs)

In [ ]:
discr_levels <-  c(2, 3, 5, 10, 15, 20, 30, 40, 50)
alpha_values <- c(0.0,0.001,0.01,0.05,0.10,0.15) # for 0.2 and more finds no biclusters
num_col_values <-c(5,10,25,50,100)


tool_out_dir <- paste0(out_dir,"xMOTIFs/")

if (!file.exists(tool_out_dir)){
    dir.create(file.path(out_dir,"xMOTIFs/"))
}

for (n_genes in g_sizes){
    for (n_pats in p_sizes){
        # read input data
        exprs_file_name <- paste0(fname_prefix,n_genes,",",n_pats,fname_suffix)
        exprs <- as.matrix(read.table(paste0(exprs_dir ,exprs_file_name),sep="\t", row.names=1, header=TRUE))
        colnames(exprs) <- seq(0,dim(exprs)[2]-1)
        basename <- gsub(".exprs.tsv","",exprs_file_name)
        network_file <- paste0(network_dir,basename,".tab")
        
        cat(basename ,"\n")
        flush.console()
        Sys.sleep(1)

        for (discr_level in discr_levels ){ 
            for (alpha in alpha_values ){ 
                for (ns in num_col_values){

                    params_outdir <- paste0("discr_levels=",discr_level,".alpha=",alpha,".ns=",ns,"/")
                    cat(params_outdir,"\n")
                    flush.console()
                    Sys.sleep(1)
                    if (!file.exists(paste0(tool_out_dir,params_outdir))){
                        dir.create(file.path(tool_out_dir,params_outdir))
                    }
                    
                    try(runXMotifs(exprs, basename,out_dir=paste0(tool_out_dir,params_outdir),
                               discr_levels=discr_level,ns=ns,nd=10,sd=5,alpha=alpha,n_runs = n_runs,number=expected_N))
                }
            }
        }
    
    }
}

## biclust - Cheng & Church
Parameter ranges selected based on Eren et al. paper
 - delta [0.5] Maximum of accepted score. [0.1-1.0] 
 - alpha [1.5] Scaling factor  [1-1.5]

In [ ]:
runChengChurch <- function(exprs, basename,out_dir="./", delta = 1.0, alpha=1.5, number=10){
    params <- paste0("delta=",delta,",alpha=",alpha)

    t0 <- as.numeric(Sys.time())
    result <- biclust(exprs, method=BCCC(), alpha=alpha, delta=delta, number=number) 
    runtime <- as.numeric(Sys.time()) - t0
    runtime <- round(runtime,2)

    # write output
    out_file <- paste0(out_dir,basename,".ChengChurch.biclust_results.txt")
    description <- paste0("Sample_id:",basename,"; Parameters:",params,"; Runtime:",runtime)
    if (result@Number>0){
        biclust::writeBiclusterResults(out_file, result, description,
                                   rownames(exprs), colnames(exprs), append=FALSE, delimiter="\t")
    }
    else{
        print("\t\tno biclusters found ")
    }
}

suppressWarnings(runChengChurch(exprs, basename,out_dir=out_dir, delta = 0.1, alpha=0.9, number=expected_N))

In [ ]:
alpha_values <- seq(1.0,1.5,0.1) 
delta_values <- seq(0.1,1.0,0.1)

In [ ]:
tool_out_dir <- paste0(out_dir,"ChengChurch/")
if (!file.exists(tool_out_dir)){
    dir.create(file.path(out_dir,"ChengChurch/"))
}

for (n_genes in g_sizes){
    for (n_pats in p_sizes){
        # read input data
        exprs_file_name <- paste0(fname_prefix,n_genes,",",n_pats,fname_suffix)
        exprs <- as.matrix(read.table(paste0(exprs_dir ,exprs_file_name),sep="\t", row.names=1, header=TRUE))
        colnames(exprs) <- seq(0,dim(exprs)[2]-1)
        basename <- gsub(".exprs.tsv","",exprs_file_name)
        network_file <- paste0(network_dir,basename,".tab")
        
        message(paste0(basename,"\n"))
        flush.console()
        Sys.sleep(1)
        for (alpha in alpha_values){
            for (delta in delta_values){
                params_outdir <- paste0("alpha=",alpha,".delta=",delta,"/")
                cat(paste0("\talpha=",alpha,"; delta=",delta),"\n")
                flush.console()
                Sys.sleep(1)
                if (!file.exists(paste0(tool_out_dir,params_outdir))){
                    dir.create(file.path(tool_out_dir,params_outdir))
                }
                suppressWarnings(try(runChengChurch(exprs, basename,out_dir=paste0(tool_out_dir,params_outdir),
                               delta = delta, alpha=alpha, number=expected_N)))
            }
        }
        
    }
}


## Fabia
 - p - number of hidden factorss = number of biclusters; p=1000 -> too many bicusters
 - alpha - sparseness loadings (0 - 1.0); default = 0.01 [0.001,0.005,0.01,0.05,0.1,0.5]
 - data normalization=0 and center=0 - no normalization or centering
 - spl - sparseness prior loadings (0 - 2.0); default = 0 (Laplace).
 - spz	 - sparseness factors (0.5 - 2.0); default = 0.5 (Laplace).

In [ ]:
list2str <- function(x){
    x2 <- unlist(x)
    if(is.numeric(x2)){
    x2<-as.double(x2)
    }
    return(unlist(gsub(",","",toString(x2))))
}

write_ut_file,append=append )
    cat(deFabia_results <- function(result,descr,out_file,append = FALSE){
    sink(oscr,"\n",append=append )
    df <- as.data.frame(extractBic(result)$bic)
    df$binp <- lapply(df$binp,list2str)
    df$bixn <- lapply(df$bixn,list2str)
    df$biypn <- lapply(df$biypn,list2str)
    for (i in seq(1,dim(df)[1])){
        cat(paste(unlist(df$binp[i]), collapse = "\t"),"\n",append=append )
        cat(paste(unlist(df$bixn[i]), collapse= "\t"),"\n",append=append )
        cat(paste(unlist(df$biypn[i]), collapse= "\t"),"\n",append=append)
    }
    sink()
}


runFabia <- function(exprs, basename,out_dir="./",number = 10,alpha=0.01,spl=0,spz=0.5, n_runs = 10){
    append<-FALSE
    for (i in seq(1,n_runs)){
        params <- paste0("alpha=",as.character(alpha),",spl=",spl,",spz=",spz)

        t0 <- as.numeric(Sys.time())
        result <- suppressMessages(fabia(exprs,p=number, norm=0, center=0, cyc=500, alpha=alpha, spl=spl, spz=spz))
        runtime <- as.numeric(Sys.time()) - t0
        runtime <- round(runtime,2)

        # write output
        out_file <- paste0(out_dir,basename,".Fabia.biclust_results.txt")
        description <- paste0("Sample_id:",basename,"; Parameters:",params,"; Runtime:",runtime,"; Run:",i)
        write_Fabia_results(result,description,out_file,append=append)
        append<-TRUE
    }
}

runFabia(exprs, basename,out_dir=out_dir,number = 10,alpha=0.01,spl=0,spz=0.5, n_runs = 2)


In [ ]:
alpha_values <- c(0.001,0.005,0.01,0.05,0.1,0.5)
spl_values <- c(0,0.5,1.0,1.5,2.0)
spz_values <- c(0,0.5,1.0,1.5,2.0)

tool_out_dir <- paste0(out_dir,"FABIA/")

if (!file.exists(tool_out_dir)){
    dir.create(file.path(out_dir,"FABIA/"))
}

for (n_genes in g_sizes){
    for (n_pats in p_sizes){
        # read input data
        exprs_file_name <- paste0(fname_prefix,n_genes,",",n_pats,fname_suffix)
        exprs <- as.matrix(read.table(paste0(exprs_dir ,exprs_file_name),sep="\t", row.names=1, header=TRUE))
        colnames(exprs) <- seq(0,dim(exprs)[2]-1)
        basename <- gsub(".exprs.tsv","",exprs_file_name)
        network_file <- paste0(network_dir,basename,".tab")
        
        cat(basename ,"\n")
        flush.console()
        Sys.sleep(1)
        for (alpha in alpha_values){
            for (spl in spl_values){
                for (spz in spz_values){
                    
                    params_outdir <- paste0("alpha=",alpha,".spl=",spl,".spz=",spz,"/")
                    cat(paste0("\talpha=",alpha,"; spl=",spl,"; spz=",spz),"\n")
                    flush.console()
                    Sys.sleep(1)
                    if (!file.exists(paste0(tool_out_dir,params_outdir))){
                        dir.create(file.path(tool_out_dir,params_outdir))
                    }
                    suppressMessages(try(runFabia(exprs, basename,out_dir=paste0(tool_out_dir,params_outdir),
                                                  number = expected_N,alpha=alpha,spl=spl,spz=spz, n_runs = 10)))
                }
            }
        }
    }
}

## ISA 


* row and column thresholds for mean over rows and columns - ISA automatically selects the most appropriate values in given ranges; [not sure whether to use negative thresholds for down-regulation?; the program throws warning]
* number of seeds - we suggest trying 5-50% of columns; runtime strongly depends on no.seeds

In [ ]:
runISA <- function(exprs, basename,out_dir="./",no_seeds=10, thresholds=seq(0.5,2.0,by=0.5) ,n_runs = 3){
    append<-FALSE
    # write output
    out_file <- paste0(out_dir,basename,".ISA.biclust_results.txt")
    for (i in seq(1,n_runs)){
        
        params <- paste0("no.seeds=",no_seeds)
        t0 <- as.numeric(Sys.time())
        up <- isa2::isa(as.matrix(exprs),thr.row=thresholds, thr.col=thresholds,
                        no.seeds=no_seeds,direction=c("up", "up"))
        down <- isa2::isa(as.matrix(exprs),thr.row=thresholds, thr.col=thresholds,
                        no.seeds=no_seeds,direction=c("down", "down"))
        result <- isa.biclust(c(up,down))
        runtime <- as.numeric(Sys.time()) - t0
        runtime <- round(runtime,2)

        description <- paste0("Sample_id:",basename,"; Parameters:",params,"; Runtime:",runtime,"; Run:",i)
        if (result@Number>0){
            biclust::writeBiclusterResults(out_file, result, description,
                                       rownames(exprs), colnames(exprs), append=append, delimiter="\t")
            append<-TRUE
        }
        else{
            print(paste0("\t\tno biclusters found in run ",i,direction ))
        }
    }
}
runISA(exprs, basename,out_dir=out_dir,no_seeds=50,n_runs = 2)

In [ ]:
thresholds <- seq(0.5,2.0,by=0.5)
no_seeds_values <- seq(0.05,0.5,by=.05)

tool_out_dir <- paste0(out_dir,"ISA/")

if (!file.exists(tool_out_dir)){
    dir.create(file.path(out_dir,"ISA/"))
}

for (n_genes in g_sizes){
    for (n_pats in p_sizes){
        # read input data
        exprs_file_name <- paste0(fname_prefix,n_genes,",",n_pats,fname_suffix)
        exprs <- as.matrix(read.table(paste0(exprs_dir ,exprs_file_name),sep="\t", row.names=1, header=TRUE))
        colnames(exprs) <- seq(0,dim(exprs)[2]-1)
        basename <- gsub(".exprs.tsv","",exprs_file_name)
        network_file <- paste0(network_dir,basename,".tab")
        
        message(basename ,"\n")
        flush.console()
        Sys.sleep(1)
        for (no_seeds in no_seeds_values*dim(exprs)[2]){
            
            params_outdir <- paste0("no_seeds=",no_seeds,"/")
            cat("\t",params_outdir ,"\n")
            flush.console()
            Sys.sleep(1)
            if (!file.exists(paste0(tool_out_dir,params_outdir))){
                dir.create(file.path(tool_out_dir,params_outdir))
            }
            runISA(exprs, basename,out_dir=paste0(tool_out_dir,params_outdir),no_seeds=no_seeds,n_runs = n_runs)
        }
    }
}

In [ ]:
R.home()